# Классы

## Импорты

In [1]:
from time import time
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import ForeignKey, PrimaryKeyConstraint
from sqlalchemy.orm import relationship

## База

**Объектно-ориентированное программирование** (ООП) - один из ключевых подходов к программированию. В нем программы представляются как набор объектов, каждый из которых - это представитель некоторого более общего типа (класса).

Класс описывает свойства (атрибуты) и действия (методы) объекта. Например, у нас есть словарь (не тот, который dict в питоне, а тот, который русско-английский). 

Какие у него есть свойства?
- Исходный язык (например, русский)
- Язык перевода (например, английский)
- Количество слов
- Какая-то метаинформация (стиль, составитель, год создания/последнего изменения)

А какие мы хотим уметь выполнять с ним действия?
- Получить перевод слова (возможно, в обе стороны)
- Добавить слово
- Удалить слово (а вдруг понадобиться)
- Получить список всех слов (например, чтобы использовать где-то еще)

Если говорить про код, то мы хотим, чтобы вот это работало:

Свойства (с этого момента - атрибуты):

```
# Исходный язык
my_dict.source_language
# Язык перевода
my_dict.target_language
# Количество слов
my_dict.size
# Метаинформация
my_dict.meta
```

Действия ( с этого момента - методы):

```
# Получить перевод слова
my_dict.translate(word='')
# Добавить слово
my_dict.add_word(source='', target='')
# Удалить слово 
my_dict.delete_word(word='')
# Получить список всех слов: может быть метод, а может и атрибут. В чем разница?
my_dict.get_all_words()
my_dict.vocabulary
```

Как это должно выглядеть снаружи с точки зрения пользователя, мы описали. Теперь давайте посмотрим, как наш словарь будет выглядеть изнутри. Тут есть несколько моментов:
- Методы - это функции внутри класса, атрибуты - это переменные внутри класса (но не все)
- У каждого класса должен быть метод `__init__`, который отвечает за создание отдельного объекта, относящегося к этому классу (экземпляра класса). Этот метод, кроме всех переменных, которые мы хотим передать для создания, должен первым аргументом иметь `self` (вообще, назвать можно как угодно, но так не принято делать), который отвечает за работу с самим классом изнутри: например, вам надо из функции добавления слова иметь доступ к словарю.
- Классы обычно называются с большой буквы, а объекты-представители с маленькой (если вы пользуетесь PyCharm или любой другой штукой с проверкой PEP8, то они любезно вам об этом напомнят).

Теперь давайте напишем шаблон нашего словаря:

In [2]:
class Dictionary:
    
    def __init__(self, source=None, target=None, meta=None):
        # функция, которая создает наш словарь и прописывает, какие у него есть атрибуты
        self.source_language = source
        self.target_language = target
        self.meta = meta
        self.size = 0
        self.vocabulary = {}
        
    def translate(self, word):
        # переводим слово
        new_word = self.vocabulary.get(word, None)
        if new_word is None:
            print(f'{word} not in dictionary')
        return new_word
    
    def add_word(self, source, target):
        # добавляем слово
        if source not in self.vocabulary:
            self.vocabulary[source] = target
            self.size += 1
        else:
            print(f'{source} is already added')
    
    def delete_word(self, word):
        # удаляем слово
        if word in self.vocabulary:
            self.vocabulary.pop(word)
            self.size -= 1
        else:
            print(f'{word} not in dictionary')
            
    def get_all_words(self):
        # получаем все слова из словаря (в данном виде - весь словарь целиком)
        return self.vocabulary

Обернем ячейки из начала тетрадки в функцию и будем считать, что это способ тестировать наш класс.

In [3]:
def test(my_dict, src='кошка', tgt='cat'):
    print(f'Сам словарь: {my_dict}', end='\n\n')
    
    print('Атрибуты')
    print(f'Исходный язык: {my_dict.source_language}')
    print(f'Метаинформация: {my_dict.meta}', end='\n\n')
    
    print('Методы')
    print(f'Добавить слово')
    my_dict.add_word(source=src, target=tgt)
    print(f'Все слова в словаре: {my_dict.get_all_words()}')
    print(f'Количество слов: {my_dict.size}', end='\n\n')
    
    print(f'Получить перевод слова "{src}": "{my_dict.translate(word=src)}"', end='\n\n')
    
    print('Удалить слово')
    my_dict.delete_word(word=src)
    print(f'Все слова в словаре: {my_dict.get_all_words()}')
    print(f'Количество слов: {my_dict.size}')

Посмотрим, что получится, но сначала создадим сам словарь

In [4]:
dct = Dictionary('ru', 'en', {'name': 'my dict'})

In [5]:
test(dct)

Сам словарь: <__main__.Dictionary object at 0x104d379a0>

Атрибуты
Исходный язык: ru
Метаинформация: {'name': 'my dict'}

Методы
Добавить слово
Все слова в словаре: {'кошка': 'cat'}
Количество слов: 1

Получить перевод слова "кошка": "cat"

Удалить слово
Все слова в словаре: {}
Количество слов: 0


Заметим, что вывесети словарь целиком у нас не очень получилось. Это происходит из-за того, что питон не имеет какого-то способа по умолчанию, чтобы показывать пользовательские классы. Но дальше, если успеем, я скажу, как это можно сделать.

А теперь давайте еще немного посмотрим теории про классы

## Три кита ООП

- **Инкапсуляция** - создание разного уровня доступа внутри класса (доступные все атрибуты или скрытые) - это уже следующий уровень
- **Наследование** - можем перенимать атрибуты и методы у другого класса (наследовать их)
- **Полиморфизм** - можем использовать одинаковый интерфейс у разных объектов (основано на наследовании)

### Инкапсуляция

Когда мы писали класс словаря, мы подразумевали, что пользователь будет что-то добавлять или удалять с помощью специальных методов. Вот только, пользователям чаще всего не очень интересно, что мы подразумевали. Здесь возникает "первый кит" ООП: мы хотим, чтобы снаружи было доступно только то, что изначально было на это рассчитано.

In [6]:
# вот так
dct.add_word('мяч', 'ball')
# а не вот так
dct.vocabulary['питон'] = 'python'

In [7]:
dct.get_all_words()

{'мяч': 'ball', 'питон': 'python'}

К сожалению, у питона конкретно с этим принципом все не очень хорошо: чем глубже вы что-то спрячете, тем больше букв пользователю придется написать, чтобы это достать, но он все равно однажды достанет

Тем не менее, у питона есть несколько методов что-то спрятать, я скажу только о самых простых.

Во-первых, можно поставить один или два знака "_" в начале названия атрибута или метода, и тогда питон либо просто намекнет, что пользователь не прав, либо выдаст ошибку, когда тот попытается обратиться к ним. Перепишем наш класс словаря, сделав атрибут со словами скрытым и добавив какой-нибудь скрытый метод.

In [8]:
class SafeDictionary:
    
    def __init__(self, source=None, target=None, meta=None):
        self.source_language = source
        self.target_language = target
        self.meta = meta
        self.size = 0
        # вот он атрибут, который мы прячем
        self.__vocabulary = {}
        # а вот вызов метода, который снаружи недоступен
        self.__detect_creation()
        
    def translate(self, word):
        new_word = self.__vocabulary.get(word, None)
        if new_word is None:
            print(f'{word} not in dictionary')
        return new_word
    
    def add_word(self, source, target):
        if source not in self.__vocabulary:
            self.__vocabulary[source] = target
            self.size += 1
        else:
            print(f'{source} is already added')
    
    def delete_word(self, word):
        if word in self.__vocabulary:
            self.__vocabulary.pop(word)
            self.size -= 1
        else:
            print(f'{word} not in dictionary')
            
    def get_all_words(self):
        return self.__vocabulary
    
    def __detect_creation(self):
        self.creation_time = time()

In [9]:
dct = SafeDictionary('ru', 'en', {'name': 'my dict'})

Теперь питон на нас ругается при попытке достать словарь на прямую, нам придется пользоваться методом, который мы сами и написали.

In [10]:
dct.__vocabulary

AttributeError: 'SafeDictionary' object has no attribute '__vocabulary'

In [11]:
dct.get_all_words()

{}

Но мы все еще можем его немного обмануть

In [12]:
dct._SafeDictionary__vocabulary

{}

Теперь посмотрим, что у нас с методом (все то же самое в целом)

In [13]:
dct.__detect_creation()

AttributeError: 'SafeDictionary' object has no attribute '__detect_creation'

In [14]:
dct._SafeDictionary__detect_creation()

Вообще, есть в питоне приняты три вида названий атрибутов и методов:
- Просто названия с маленькой буквы (public): доступны снаружи и формируют интерфейс для пользователя
- Названия, которые начинаются с "_" (protected): доступны пользователю, но мы мягко намекаем, что это можно трогать только, если он очень уверен в своих действиях
- Названия с "__" (private): недоступны снаружи, но их все еще можно вытащить методом хитрых комбинаций (пользователь точно знает, что лезет куда-то не туда)

### Наследование

Представьте, что у нас есть словарь, с которым все просто: слова всегда подаются по одному. Но вдруг нам понадобилось добавлять слова списками (и апельсины бочками, да). Здесь нам может помочь наследование.

Общая идея заключается в том, что мы не хотим переписывать общие для старого и нового словаря методы. Тот же вывод всех слов или фиксацию времени. Но мы хотим сделать новый метод, который умеет добавлять список. Для этого можно создать новый класс, но унаследовать его от предыдущего

In [15]:
class DictionaryWithList(SafeDictionary):

    def add_many_words(self, words):
        # добавляем много слов
        for word, target in words:
            self.add_word(word, target)
            
    def delete_many_words(self, words):
        # удаляем много слов
        for word in words:
            self.delete_word(word)

In [16]:
dct = DictionaryWithList('ru', 'en', {'name': 'my dict'})

In [17]:
dct.add_word('кошка', 'cat')

In [18]:
words = [('собака', 'dog'), ('птица', 'bird'), ('стол', 'table')]
dct.add_many_words(words)
dct.get_all_words()

{'кошка': 'cat', 'собака': 'dog', 'птица': 'bird', 'стол': 'table'}

In [19]:
words = ['птица', 'стол']
dct.delete_many_words(words)
dct.get_all_words()

{'кошка': 'cat', 'собака': 'dog'}

### Полиморфизм

Теперь представим, что нам нужен словарь, который умеет сам приводить слово к начальой форме и только после этого переводить его. При этом, нам бы очень не хотелось объяснять пользователям, что теперь им нужно использовать метод с другим именем.

Здесь нам поможет полиморфизм, идея которого в том, что под одинаковым интерфейсом может скрываться разное содержание, реализующее нужный нам метод. И мы все еще не переопределяем то, что менять не надо.

In [20]:
class SuperDictionary(DictionaryWithList):
    
    def translate(self, word):
        # переводим слово, получив перед этим его лемму
        word = morph.parse(word)[0].normal_form
        # здесь мы вызываем метод перевода из родителя, так как здесь мы его уже переопределили
        new_word = DictionaryWithList.translate(self, word)
        return new_word

In [21]:
dct = SuperDictionary('ru', 'en', {'name': 'my dict'})

In [22]:
words = [('собака', 'dog'), ('птица', 'bird'), ('стол', 'table')]
dct.add_many_words(words)
dct.get_all_words()

{'собака': 'dog', 'птица': 'bird', 'стол': 'table'}

In [23]:
dct.translate('собаки')

'dog'

## Дополнительная информация

- Статические методы: НЕ принимают в качестве первого аргумента сам объект

In [24]:
class StrangeDictionary(Dictionary):
    
    @staticmethod
    def strange_method(word):
        print(word, len(word))
        
    # если не указать, что метод static
    def very_strange_method(word):
        print(word, len(word))

In [25]:
dct = StrangeDictionary()

In [26]:
dct.strange_method('кошка')

кошка 5


In [27]:
dct.very_strange_method('кошка')

TypeError: StrangeDictionary.very_strange_method() takes 1 positional argument but 2 were given

In [28]:
dct.very_strange_method()

TypeError: object of type 'StrangeDictionary' has no len()

- *args, **kwargs: позволяют собрать все аргументы функции в один кортеж и(ли) словарь:

In [29]:
def func(*args, **kwargs):
    print(f'args: {args}')
    print(f'kwargs: {kwargs}')

In [30]:
func(1, 2, 3, 5, a=23, b=12, c=74)

args: (1, 2, 3, 5)
kwargs: {'a': 23, 'b': 12, 'c': 74}


- Методы с названием имени \_\_METHOD\_\_: нужны для переопределения базовых операций:

Например, `__repr__` позволяет рассказать питону, как надо отображать объект этого класса, а `__str__` - как надо его переводить в строку (да, это разные вещи) 

In [31]:
class BeautifulDictionary(SuperDictionary):
    def __repr__(self):
        s = SuperDictionary.__name__.split('.')[-1]
        args = []
        for el, val in self.__dict__.items():
            args.append(f'{el}={val}')
        if len(args) > 0:
            args = f'({", ".join(args)})'
            s += args
        return s

In [32]:
dct = BeautifulDictionary('ru', 'en', {'name': 'my dict'})

In [33]:
dct

SuperDictionary(source_language=ru, target_language=en, meta={'name': 'my dict'}, size=0, _SafeDictionary__vocabulary={}, creation_time=1669304796.154612)

- Что на самом деле делает питон, когда скрывает атрибуты и методы: мухлюет, как всегда

Питон вместе того, чтобы сохранить атрибут с тем именем, которое вы ему дали, сохраняет его в виде `_CLASS__ATTRIBUTE`

In [34]:
dct.__dict__

{'source_language': 'ru',
 'target_language': 'en',
 'meta': {'name': 'my dict'},
 'size': 0,
 '_SafeDictionary__vocabulary': {},
 'creation_time': 1669304796.154612}

# Flask

А теперь о том, зачем это было именно здесь. Вы уже работали с базой данных через питон и, наверное, должны были заметить, насколько легко ошибиться в запросах: сделать опечатку, неправильно переписать имя таблицы или столбка6, наруть синтаксис запросов... Эту историю можно упростить, если использовать классы как способ обращения к базе данных. 

Для того, чтобы понять, как это делать, попробуем подключить базу данных к нашему приложению и сделать сайт про фильмы.

Вот [тут](https://github.com/hse-ling-python/seminars/tree/master/flask_applications/imdb_site) лежит шаблон сайта, его можно использовать.

Для удобной работы с базами данных и фласком одновременно была придумана библиотека, которая создает удобную интеграцию одного в другое.
Вообще, кроме самого фласка, нам нужны сдедующие библиотеки:
`sqlalchemy`
`flask_sqlalchemy`

**Сначала посмотрим, как объяснить питону строение базы**

Обычно, все описания кладут в отдельный файл, из которого их потом импортируют. Это в целом хороший подход к любому проекту. В нашем случае все модели лежат в файле `models.py`. там мы сначала создаем объект, который отвечает за интерфейс работы с базой, а потом описываем каждую таблицу. 

In [35]:
# models.py

from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy()

# дальше описания таблиц

Каждая таблица описывается отделно: с помощью ```__tablename__``` передается ее имя, а названия столбиков совпадают с названиями в нашей таблице. Еще используется специальная "обертка" для столбца, где можно прописать, какой это типа данных и если это первиный ключ, то на это указать.

Ниже пример класса, описывающего одну из таблиц.

In [36]:
class Person(db.Model):
    __tablename__ = "people"

    # атрибут person_id будет хранить значение столбца id, это целое число и первичный ключ
    person_id = db.Column('person_id', db.Integer, primary_key=True)
    name = db.Column('name', db.Text)
    born = db.Column('born', db.Integer)
    died = db.Column('died', db.Integer)

    films = relationship("Film", secondary='crew')

Можно указать, что столбец ссылается на ключ из другой таблицы, например, как у нас в crew. Для этого используется `ForeignKey`

In [37]:
class Crew(db.Model):
    __tablename__ = 'crew'
    __table_args__ = (PrimaryKeyConstraint('title_id', 'person_id'),)

    film_id = db.Column('title_id', db.Integer, ForeignKey('titles.title_id'))
    person_id = db.Column('person_id', db.Integer, ForeignKey('people.person_id'))

Мы можем сделать так, чтобы к фильму привязывались люди, которые там участвуют (в атрибуте будет лежать список).

Для этого мы используем `relationship`. Так как мы здесь соединяем две таблицы через вспомогательную, то мы это делаем, указав, с каким объектом мы соединяем и какая вспомогательная таблица.

Для рейтинга мы просто связываем две таблицы, где связь один-один, мы можем это сделать сразу, в primaryjoin указываем, как мы свзяываем таблицы, то есть какие столбцы соответствуют друг другу.

In [38]:
class Film(db.Model):
    __tablename__ = "titles"

    # имя колонки = специальный тип (тип данных, первичный ключ)
    film_id = db.Column('title_id', db.Integer, primary_key=True)

    rating = db.relationship('Rating', uselist=False, primaryjoin="Film.film_id==Rating.film_id")

    ...
    
    crew = db.relationship("Person", secondary='crew') # с кем, через какую таблицу

**Как подключить базу к приложению**


Мы должны импортировать фласк и использовать модуль `flask-sqlalchemy`, который позволяет работать с базой. Главное - это прописать путь к базе - там есть часть ```sqlite3:///```- это обозначение для того, что мы работает с таким типом базы, а дальше путь внутри проекта

Пусть база лежит там же в папке. Тогда в файле приложения мы импортируем объект базы6 который создали в `models.py` и подключаем к приложению.

In [39]:
from flask import Flask

# создаем приложение
app = Flask(__name__)

# подключаем нашу базу sqlite:/// - это тип базы, потом просто имя файла, который лежит в той же папке
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///imdb_small_indexed.db'
# чтобы ок работать с изменениями в базе (коммиты и обновления информации, если такие есть)
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False 

# указываем базе, какое наше приложение
db.app = app
db.init_app(app)